In [1]:
import numpy as np
import pickle
from collections import Counter
from tqdm import tqdm

In [74]:
with open(r'D:\dstore\nlp\w2v\wcounts', 'rt', encoding='utf8') as f:
    wlst = Counter({f'_{x.strip().split()[1]}_': int(x.strip().split()[0]) for x in f.readlines() if int(x.strip().split()[0]) > 9})

with open(f'D:/dstore/nlp/w2v/uwlst', 'rb') as f:
    cnts = pickle.load(f)

with open(f'D:/dstore/nlp/w2v/bwlst', 'rb') as f:
    bcnts = pickle.load(f)

affs = set()

with open(r'D:\dstore\nlp\w2v\prefs2', 'rt', encoding='utf8') as f:
    affs = {x.strip() for x in f.readlines()}

with open(r'D:\dstore\nlp\w2v\suffs2', 'rt', encoding='utf8') as f:
    for x in f.readlines():
        affs.add(x.strip())


In [ ]:
[x for x in bcnts.most_common() if len(x[0]) == 4 and x[0][0] == '_']

In [77]:

def fragment(word: str):
    queue = []
    finished = []
    keys = []
    esuf = ('d_', 'r_', 's_', 'n_', "'d_", "'s_", "'n_", "'t_", 'er_', 'ed_', 'es_', 'or_')
    doubles = ('ll', 'tt', 'pp', 'mm', 'nn', 'bb', 'ss', 'dd')
    ysufs = ('ies_', 'iful_', 'iness_', 'ities_')
    for suf in esuf:
        if suf in word:
            k = ([suf], word.replace(suf, '_'))
            keys.append(k)
            queue.append((*k, affs.copy()))
            finished.append((*k, affs.copy()))
    for item in queue[::-1]:
        for suf in esuf:
            if suf in item[1]:
                k = (sorted([*item[0], suf]), item[1].replace(suf, '_'))
                if k not in keys:
                    keys.append(k)
                    queue.append((*k, item[2]))
                if k not in finished:
                    finished.append(k)
    queue.append(([], word, affs.copy()))
    for i, x in enumerate(queue):
        if x[1][1:3] in doubles:
            mod = list(x[1])
            mod.pop(1)
            queue[i] = (x[0], ''.join(mod), x[2])
        if x[1][-3:-1] in doubles:
            mod = list(x[1])
            mod.pop(-2)
            queue[i] = (x[0], ''.join(mod), x[2])
    while queue:
        item = queue.pop(0)
        found = False
        for affix in item[2]:
            if affix in item[1]:
                found = True
                if affix in ysufs:
                    k = (sorted([*item[0], affix]), item[1].replace(affix, 'y_'))
                else:
                    k = (sorted([*item[0], affix]), item[1].replace(affix, '_'))
                if k not in keys:
                    if k[1][1:3] in doubles:
                        mod = list(k[1])
                        mod.pop(1)
                        k = (k[0], ''.join(mod))
                    if k[1][-3:-1] in doubles:
                        mod = list(k[1])
                        mod.pop(-2)
                        k = (k[0], ''.join(mod))
                    aff_copy = item[2].copy()
                    keys.append(k)
                    aff_copy.remove(affix)
                    queue.append((*k, aff_copy))
                if k not in finished:
                    finished.append(k)
        
        if not found and (item[0], item[1]) not in keys:
            finished.append((item[0], item[1]))
    print(word)
    for x in finished[::-1]:
        print(x[0], x[1])
        """ if x[1] not in wlst or x[1] != '__':
            finished.remove(x)
        else:
            print(x[0], x[1]) """
    print('\n')
    return


In [79]:
tword = '_ill_'
fragment(tword)

_ill_
['_i'] _l_




In [81]:
c = np.random.randint(0, len(wlst) - 101)
test = [fragment(x[0]) for x in wlst.most_common()[c:c+20]]

_toxodon_
[] _toxodon_
['n_'] _toxodo_


_tragicomedies_
['ies_', 'y_'] _tragicomed_
['ies_'] _tragicomedy_
['es_'] _tragicomedi_
['s_'] _tragicomedie_


_transmogrified_
['_trans'] _mogrified_
['_trans', 'ed_'] _mogrifi_
['_trans', 'd_'] _mogrifie_
['ed_'] _transmogrifi_
['d_'] _transmogrifie_


_transship_
['_trans', 'ship_'] __
['ship_'] _trans_
['_trans'] _ship_


_treasuring_
['ing_'] _treasur_


_trematoda_
[] _trematoda_


_trochodendron_
[] _trochodendron_
['n_'] _trochodendro_


_tropicalization_
['ation_'] _tropicaliz_
['ion_'] _tropicalizat_
['tion_'] _tropicaliza_
['n_'] _tropicalizatio_


_trullan_
[] _trullan_
['n_'] _trulla_


_tude_
[] _tude_


_tyrannis_
[] _tyrannis_
['s_'] _tyranni_


_umlauted_
[] _umlauted_
['ed_'] _umlaut_
['d_'] _umlaute_


_unbent_
['_be', '_un'] _nt_
['_un', 'ent_'] _b_
['_un'] _bent_
['ent_'] _unb_


_uncaria_
['_un'] _caria_


_uncarved_
['_un'] _carved_
['_un', 'ed_'] _carv_
['_un', 'd_'] _carve_
['ed_'] _uncarv_
['d_'] _uncarve_


_uncompla

In [ ]:
[x for x in wlst.most_common() if 'demo' in x[0]][:64]

In [104]:
with open(f'D:/dstore/nlp/w2v/uwlst', 'rb') as f:
    cnts = pickle.load(f)

for n in range(3, 8):
    print(np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]))

323.2823342054374
42.49057119965607
11.874224257866846
5.461737040443904
2.6007610127323284


In [106]:
for n in range(3, 8):
    print(np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]) ** (1-((n * 0.01) * 2.667)))

203.60588166144703
28.4830385751262
8.537026649600872
4.162393305015124
2.1757317224699224


In [101]:
for n in range(3, 8):
    scale = np.mean([x[1] for x in cnts.most_common() if len(x[0]) == n]) ** (1-((n * 0.01) * 2.667))
    for x in cnts:
        if len(x) == n:
            cnts[x] = int(cnts[x] / scale)

In [109]:
cnts.most_common()

[('ing', 12981),
 ('ter', 11481),
 ('ati', 9572),
 ('ers', 8302),
 ('ent', 7746),
 ('eri', 7213),
 ('ate', 6982),
 ('lin', 6805),
 ('tin', 6661),
 ('ill', 6585),
 ('ari', 6466),
 ('ine', 6317),
 ('ist', 6212),
 ('ant', 6166),
 ('ali', 6059),
 ('and', 5940),
 ('ell', 5934),
 ('oni', 5743),
 ('all', 5632),
 ('per', 5609),
 ('ste', 5600),
 ('tra', 5370),
 ('est', 5168),
 ('rin', 5153),
 ('ran', 5057),
 ('ver', 5020),
 ('lan', 5017),
 ('der', 4950),
 ('nes', 4925),
 ('man', 4822),
 ('nti', 4733),
 ('sta', 4707),
 ('tic', 4688),
 ('sti', 4684),
 ('les', 4640),
 ('rat', 4636),
 ('che', 4607),
 ('ion', 4546),
 ('ica', 4531),
 ('ian', 4523),
 ('ess', 4512),
 ('men', 4492),
 ('ani', 4487),
 ('sto', 4478),
 ('era', 4465),
 ('the', 4414),
 ('iti', 4395),
 ('tor', 4391),
 ('tio', 4367),
 ('her', 4342),
 ('res', 4281),
 ('lli', 4262),
 ('nte', 4240),
 ('ere', 4209),
 ('ast', 4184),
 ('str', 4181),
 ('tro', 4137),
 ('nde', 4107),
 ('lle', 4092),
 ('ina', 4053),
 ('chi', 4047),
 ('tri', 4043),
 ('ori

In [113]:
[x for x in bcnts.most_common() if len(x[0]) > 2] 

[('ed_', 33504675437),
 ('es_', 31894271505),
 ('gs_', 29421500253),
 ('ion_', 24148880731),
 ('rs_', 23946264269),
 ('ing_', 18052533248),
 ('ts_', 17754915160),
 ('er_', 17449354205),
 ('al_', 13947617249),
 ('gly_', 12954993901),
 ('ess_', 12137677568),
 ('ry_', 11643253227),
 ('_pr', 11035724209),
 ('tic_', 11019669081),
 ('_re', 10937013725),
 ('us_', 10908189536),
 ('ic_', 9906050175),
 ('ne_', 9278472490),
 ('_gr', 8980396363),
 ('ia_', 8949899949),
 ('_au', 8804491531),
 ('wn_', 8779589473),
 ('ation_', 8675935498),
 ('_st', 8653276265),
 ('ions_', 8522784700),
 ('_ch', 7882542705),
 ('ers_', 7504962137),
 ('_br', 7392735994),
 ('_sp', 7131192778),
 ('_mo', 6956155728),
 ('_pho', 6459346614),
 ('gton_', 6298588751),
 ('_tr', 6213817066),
 ('ate_', 6193549088),
 ('_qu', 6175490044),
 ('_ma', 6084592940),
 ('ism_', 5854612017),
 ('_un', 5776147107),
 ('ger_', 5749835598),
 ('_an', 5722199986),
 ('ter_', 5660897840),
 ('sis_', 5636810396),
 ('is_', 5189190029),
 ('or_', 5074435238